In [1]:
import pandas as pd 
import yaml
import notebook_utils as nu

In [3]:
#load the files
with open('cand_parsing.yaml') as f:
    conf_dict = yaml.safe_load(f)
language_list = nu.read_csv_to_list (conf_dict['LANGUAGE_PATH'])
df = pd.read_csv(conf_dict['TABLE_PATH'], usecols = conf_dict['COLS'].keys())

In [4]:
n_cols =df.shape[1]
df_IT = df[df['category']=='Information Technology']
df_IT = df_IT[df_IT['Experience'].notna()]
df_IT.insert(n_cols, 'entry_job', ~df_IT['Experience'].str.contains('|'.join(conf_dict['SENIOR_KEYWORDS'])))
df_IT_start = df_IT[df_IT['entry_job']==True]
df_IT_start['skills'] = df_IT_start["Experience"].map(lambda x: extract_skills(str(x), language_list))
df_IT_start['soft_skills'] = df_IT_start['skills'].map(lambda x: x[0])
df_IT_start['hard_skills'] = df_IT_start['skills'].map(lambda x: x[1])
df_IT_start['languages'] = df_IT_start['skills'].map(lambda x: x[2])
#add column holders
df_IT_start.insert(n_cols, 'location_country', ['Switzeland']*len(df_IT_start))
df_IT_start.insert(n_cols, 'preffered_name', ['John Smith']*len(df_IT_start))
df_IT_start.insert(n_cols, 'location_city', ['Geneva']*len(df_IT_start))
df_IT_start.insert(n_cols, 'last_country', ['Switzerland']*len(df_IT_start))
df_IT_start.insert(n_cols, 'education', ['Switzeland']*len(df_IT_start))
df_IT_start.insert(n_cols, 'work_permission_CH', ['True']*len(df_IT_start))
df_IT_start.insert(n_cols, 'date_of_birth', ['20.07.1985']*len(df_IT_start))
df_IT_start.insert(n_cols, 'linkedin', ['www.linkedin.com']*len(df_IT_start))
df_IT_start.insert(n_cols, 'github', ['www.github.com']*len(df_IT_start))
df_IT_start[['first_name', 'last_name']] = df_IT_start['name'].str.split(n=1, expand = True)
#rename columns
df_IT_start.rename(mapper = conf_dict['COLS'], axis = 1, inplace = True)
#drop columns out of interest
df_IT_start.drop(columns = ['skills', 'entry_job', 'name'], inplace = True)
#drop index
df_IT_start.reset_index(drop = True, inplace = True)
#save to scv
df_IT_start.to_csv('IT_candidates.csv')